Clase para hacer un scraping de Busca cursos de la Universidad Catolica de Chile

In [ ]:
#LIBRERIAS USADAS

#!pip install beautifulsoup4
from bs4 import BeautifulSoup #web scrapping
import requests
import smtplib # se utiliza para enviar mail


class ScrappingBanner:
  def __init__(self, url):
    self.url = url
    self.soup = None
    self.message = None
    self.name_course = []
    self.name_profesor = []
    self.old_course_vacancies = []
    self.new_course_vacancies = []
    self.send_messange = False # False: no envia mail ; True: envia mail

    self.auxiliar_message = None # ESTO ES UN PARCHE PARA DARLE FUNCIONALIDAD  (BORRAR)
    
    self.query_request()

    #Se crean los valores inciciales de las listas
    num_courses = int(len(self.soup.find_all('td', {'style':'font-size:13px;text-align:left;'})) / 4)
    for element in range(num_courses):
      self.name_course.append(None)
      self.name_profesor.append(None)
      self.old_course_vacancies.append(None)
      self.new_course_vacancies.append(None)

  def query_request(self):
    # hacemos requests del url, esto nos retornará un objeto "Response"
    page = requests.get(self.url)
    # se construye el objeto con el contenido de la página web
    self.soup = BeautifulSoup(page.content, 'html.parser')

  def get_name_course(self, text_name_course):
    name_text = str(text_name_course)
    inicio = name_text.find('title="')
    fin = name_text.find('"><div')
    return name_text[inicio + 7: fin]


  def get_name_profesor(self, text_name_course):
    name_text = str(text_name_course)
    inicio = name_text.find('profesor=') # se parte del puesto 9
    fin = name_text.find('#resultados')
    proto_name = name_text[inicio + 9: fin]
    return proto_name.replace("+"," ")

  def get_course_vacancies(self, text):
    text = str(text)
    inicio = text.find('>')
    fin = text.find('</')
    return text[inicio + 1: fin]

  def reload(self):
    ''' esta funcion volvera a realizar el request pero sin ajustar largos de lista listas -- DUDA: que pasa si agregar nuevas secciones??? se considerará??'''
    self.query_request()
    self.data_processing()

  def data_processing(self):
    #buscamos los nombres de los cursos
    y = self.soup.find_all('td', {'style':'font-size:13px;text-align:left;'})
    #posicion de las vacantes
    x = self.soup.find_all('td', {'style':'font-size:13px;text-align:center;'})

    n_recorridos_x = int(len(x)/7) # recordar que cada curso tiene 7 elementos
    for i in range(n_recorridos_x):
      course_vacancies_aux = int(self.get_course_vacancies(x[4+7*i]))
      if self.new_course_vacancies[i] != course_vacancies_aux:
        self.new_course_vacancies[i] = course_vacancies_aux
        self.send_messange = True


    n_recorridos_y = int(len(y)/4)
    for j in range(n_recorridos_y):
      name_course_aux = self.get_name_course(y[0+4*j])
      # self.name
      if self.name_course[j] != name_course_aux: 
        self.name_course[j] = name_course_aux
        self.send_messange = True

      name_profesor_aux = self.get_name_profesor(y[2+4*j])
      if self.name_profesor[j] != name_profesor_aux: 
        self.name_profesor[j] = name_profesor_aux
        self.send_messange = True
      
    if self.send_messange:
      self.send_mail()
    # self.create_message()

  def create_message(self):
    string = ''
    string_parche = '' # SOLUCION PARCHE, ELIMINAR A POSTERIORIDAD
    for index in range(len(self.name_course)):
      string += 'El curso '+str(self.name_course[index])+' tiene '+str(self.new_course_vacancies[index])+' vacantes con el profesor '+ str(self.name_profesor[index])+'\n\n\n'
      string_parche +='sec '+str(index + 1)+': '+ str(self.new_course_vacancies[index])+'\n\n' # SOLUCION PARCHE, ELIMINAR A POSTERIORIDAD
    self.message = string
    self.auxiliar_message = string_parche # SOLUCION PARCHE, ELIMINAR A POSTERIORIDAD
  def send_mail(self): 
    
    self.send_messange = False #se retorna al valor False por defecto
    self.create_message()

    print
    # subject = 'CUPOS BANNER 2022-1'
    # msn = 'Subject: {}\n\n{}'.format(subject, self.message.encode(encoding='UTF-8',errors='strict')) # ESTA ES LA SOLUCION CORRECTA ( TODO:PROBLEMAS CON EL ENCODING)
    subject = 'CUPOS BANNER GPTI 2022-1'
    msn = 'Subject: {}\n\n{}'.format(subject, self.auxiliar_message) # SOLUCION PARCHE, ELIMINAR A POSTERIORIDAD
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login('mbs.proyect.adm@gmail.com', 'una_clave')

    # ESTRUCTURA:  (emisor, receptor, mensaje)
    server.sendmail('mbs.proyect.adm@gmail.com','mbs.proyect.adm@gmail.com', msn)
    server.quit()
    print('Correo enviado...')

In [ ]:
import time
url = 'https://buscacursos.uc.cl/?cxml_semestre=2022-1&cxml_sigla=IIC3113#resultados' #gestion de operaciones TI
web = ScrappingBanner(url)


while True:
  web.reload()
  print(web.message)
  print(web.new_course_vacancies)
  time.sleep(1)


Se han truncado las últimas 5000 líneas del flujo de salida.
El curso IIC3113 Gestión de Proyectos de Tecnologías de Información tiene 10 vacantes con el profesor V%C3%A1zquez Enrique



[0, 10]
El curso IIC3113 Gestión de Proyectos de Tecnologías de Información tiene 0 vacantes con el profesor Droguett Mario


El curso IIC3113 Gestión de Proyectos de Tecnologías de Información tiene 10 vacantes con el profesor V%C3%A1zquez Enrique



[0, 10]
El curso IIC3113 Gestión de Proyectos de Tecnologías de Información tiene 0 vacantes con el profesor Droguett Mario


El curso IIC3113 Gestión de Proyectos de Tecnologías de Información tiene 10 vacantes con el profesor V%C3%A1zquez Enrique



[0, 10]
El curso IIC3113 Gestión de Proyectos de Tecnologías de Información tiene 0 vacantes con el profesor Droguett Mario


El curso IIC3113 Gestión de Proyectos de Tecnologías de Información tiene 10 vacantes con el profesor V%C3%A1zquez Enrique



[0, 10]
El curso IIC3113 Gestión de Proyectos de Tecnolog

In [ ]:
import time

url = 'https://buscacursos.uc.cl/?cxml_semestre=2022-1&cxml_sigla=IIC2133#resultados' #estructura de datos y algoritmos

webb = ScrappingBanner(url, url2, url3)

while True:
  webb.reload()
  print(webb.message)
  print(webb.new_course_vacancies)
  time.sleep(1)

El curso IIC2133 Estructuras de Datos y Algoritmos tiene 44 vacantes con el profesor Eterovic Yadran
El curso IIC2133 Estructuras de Datos y Algoritmos tiene 120 vacantes con el profesor Por Fijar

[44, 120]
El curso IIC2133 Estructuras de Datos y Algoritmos tiene 44 vacantes con el profesor Eterovic Yadran
El curso IIC2133 Estructuras de Datos y Algoritmos tiene 120 vacantes con el profesor Por Fijar

[44, 120]
El curso IIC2133 Estructuras de Datos y Algoritmos tiene 44 vacantes con el profesor Eterovic Yadran
El curso IIC2133 Estructuras de Datos y Algoritmos tiene 120 vacantes con el profesor Por Fijar

[44, 120]
El curso IIC2133 Estructuras de Datos y Algoritmos tiene 44 vacantes con el profesor Eterovic Yadran
El curso IIC2133 Estructuras de Datos y Algoritmos tiene 120 vacantes con el profesor Por Fijar

[44, 120]
El curso IIC2133 Estructuras de Datos y Algoritmos tiene 44 vacantes con el profesor Eterovic Yadran
El curso IIC2133 Estructuras de Datos y Algoritmos tiene 120 vacant

KeyboardInterrupt: ignored